In [69]:
from decouple import config
import pandas as pd
import sys
import os
import unicodedata
sys.path.insert(0, config('PROYECTO_DIR'))
from clases.bd.conexion import  SQLServerConnector
conn = SQLServerConnector('DBGERESA')
pd.set_option('display.max_columns', None)

✅ Conexión exitosa a la base de datos 'DBGERESA'


In [70]:
def limpiar_columna(texto): 
    # Reemplazar espacios por guiones bajos
    texto_sin_espacios = texto.replace(' ', '_')
    # Quitar los puntos
    return texto_sin_espacios.replace('.', '')

archivo = config('PROYECTO_DATA') + "/otros/BASE DE DATOS  RESULTADO vph.tomamuestra   HIS AL 25 JULIO EXORTADO 30 JULIO 2025 CON DNI - copia.xlsx"

# Leer el archivo Excel
vph = pd.read_excel(archivo)
vph.columns = [limpiar_columna(col) for col in vph.columns]
#vph['DNI'] = vph['DNI'].astype(str).str.zfill(8)


vph["Hisminsa/tomamuestra/idCita"] = pd.to_numeric(vph["Hisminsa/tomamuestra/idCita"], errors="coerce").astype("Int64")
vph["Hisminsa/resultado/idCita"] = pd.to_numeric(vph["Hisminsa/resultado/idCita"], errors="coerce").astype("Int64")
tomamuestra_str = "('" + "','".join(vph["Hisminsa/tomamuestra/idCita"].astype(str)) + "')"
resultado_str = "('" + "','".join(vph["Hisminsa/resultado/idCita"].astype(str)) + "')"

#print(dni_str)
vph.head(5)

,Establecimiento_Salud,Código_de_barras,Fecha_de_la_toma,Paciente,Resultado,Resultado/Fecha_de_Entrega,Hisminsa/tomamuestra/idCita,Hisminsa/resultado/idCita,Red_Salud,Microred,Status,DNI
0,C.S. ACCHA,CUS000023627,2025-05-17,CUTIPA HANCCO ANGELICA,NEGATIVO,2025-07-08,1216006514,1246315758,CUSCO SUR,PARURO,ER Hisminsa,44152466
1,C.S. ACCHA,CUS000023626,2025-05-17,BACA CARRASCO CONSTANTINA,NEGATIVO,NaT,1216006517,<NA>,CUSCO SUR,PARURO,Laboratorio,41732184
2,C.S. ACCHA,CUS000023625,2025-05-16,VARGAS AVILES SAIDA,NEGATIVO,NaT,1216006516,<NA>,CUSCO SUR,PARURO,Laboratorio,46393913
3,C.S. ACCHA,CUS000023624,2025-05-15,DIAZ FERNANDEZ VIRGINIA,NEGATIVO,2025-06-30,1216006515,1243333096,CUSCO SUR,PARURO,ER Hisminsa,44152449
4,C.S. ACCHA,CUS000023623,2025-05-16,LOAIZA SENCIA VALENTINA,NEGATIVO,2025-06-30,1216006513,1243333097,CUSCO SUR,PARURO,ER Hisminsa,42455594


In [71]:
q= f"""SELECT 
        h.id_cita ,
       p.Numero_Documento as DNI_HIS,
       h.Fecha_Atencion AS F_ATENCION,
       CONCAT(h.Tipo_Diagnostico,' | ',h.Codigo_Item) AS Codigo_Item,
       c.Descripcion_Item,
       ISNULL(MAX(CASE WHEN h.Id_Correlativo_Lab = 1 THEN h.Valor_Lab END), '') AS LAB1,
       ISNULL(MAX(CASE WHEN h.Id_Correlativo_Lab = 2 THEN h.Valor_Lab END), '') AS LAB2,
       ISNULL(MAX(CASE WHEN h.Id_Correlativo_Lab = 3 THEN h.Valor_Lab END), '') AS LAB3,
       h.Fecha_Registro AS F_REGISTRO,
       h.Fecha_Modificacion AS F_MODIFICACION,            
       r.est_nombre AS ESTABLECIMIENTO,
       CONCAT(r.DESC_DIST,' | ', r.DESC_PROV) AS [DISTRITO | PROVINCIA],
       S.Descripcion_Sistema AS SISTEMA,
       LTRIM(RTRIM(
            COALESCE(re.Nombres_Registrador,'') + ' ' +
            COALESCE(re.Apellido_Paterno_Registrador,'') + ' ' +
            COALESCE(re.Apellido_Materno_Registrador,'')
       )) AS REGISTRADOR
FROM DBGERESA.dbo.HISMINSA h
INNER JOIN DBGERESA.dbo.MAESTRO_PACIENTE p
    ON p.Id_Paciente = h.Id_Paciente
INNER JOIN DBGERESA.dbo.RENIPRESS r
    ON r.COD_ESTAB = h.renipress
LEFT JOIN DBGERESA.dbo.MAESTRO_HIS_CIE_CPMS c
    ON c.Codigo_Item = h.Codigo_Item
LEFT JOIN DBGERESA.dbo.MAESTRO_HIS_SISTEMA S
    ON S.Id_Sistema = H.Id_AplicacionOrigen
LEFT JOIN DBGERESA.dbo.MAESTRO_REGISTRADOR re
    ON re.Id_Registrador = h.Id_Registrador
WHERE h.id_cita IN {tomamuestra_str}
  AND h.Anio = 2025
  and h.Codigo_Item IN ('87621')
  and p.Genero='F'
GROUP BY
     h.id_cita,
    p.Numero_Documento ,
    h.Fecha_Atencion,
    h.Tipo_Diagnostico,
    h.Codigo_Item,
    c.Descripcion_Item,
    h.Fecha_Registro,
    h.Fecha_Modificacion,
    S.Descripcion_Sistema,
    r.est_nombre,
    r.DESC_DIST,
    r.DESC_PROV,
    re.Nombres_Registrador,
    re.Apellido_Paterno_Registrador,
    re.Apellido_Materno_Registrador
ORDER BY h.Fecha_Atencion DESC;"""
his_toma=conn.ejecutar_sql(q,retornar_datos=True)
his_toma=pd.DataFrame(his_toma)
his_toma['id_cita'] = pd.to_numeric(his_toma["id_cita"], errors="coerce").astype("Int64")
his_toma.head(5)

,id_cita,DNI_HIS,F_ATENCION,Codigo_Item,Descripcion_Item,LAB1,LAB2,LAB3,F_REGISTRO,F_MODIFICACION,ESTABLECIMIENTO,DISTRITO | PROVINCIA,SISTEMA,REGISTRADOR
0,1249712169,47605389,2025-07-14,D | 87621,TEST PVH (TOMA DE MUESTRA),2,,,2025-07-19 09:24:15,NaT,URCOS,URCOS | QUISPICANCHI,INFORMACION REMITIDA POR CANCER,IVAN TELLO MORALES
1,1249710903,46518177,2025-07-14,D | 87621,TEST PVH (TOMA DE MUESTRA),1,,,2025-07-19 09:22:25,NaT,RONDOCAN,RONDOCAN | ACOMAYO,INFORMACION REMITIDA POR CANCER,OLINDA QUISPE DIAZ
2,1249712173,70352839,2025-07-14,D | 87621,TEST PVH (TOMA DE MUESTRA),2,,,2025-07-19 09:24:15,NaT,URCOS,URCOS | QUISPICANCHI,INFORMACION REMITIDA POR CANCER,IVAN TELLO MORALES
3,1249710892,47559570,2025-07-14,D | 87621,TEST PVH (TOMA DE MUESTRA),1,,,2025-07-19 09:22:25,NaT,QUINCE MIL,CAMANTI | QUISPICANCHI,INFORMACION REMITIDA POR CANCER,MARIZABEL VALER FIGUEROA
4,1249712205,42104351,2025-07-14,D | 87621,TEST PVH (TOMA DE MUESTRA),2,,,2025-07-19 09:24:15,NaT,ACOS,ACOS | ACOMAYO,INFORMACION REMITIDA POR CANCER,LIZ KARINA ANDRADE TITO


In [72]:
q= f"""SELECT 
        h.id_cita as id_cita_r,
       p.Numero_Documento as DNI_HIS,
       h.Fecha_Atencion AS F_ATENCION,
       CONCAT(h.Tipo_Diagnostico,' | ',h.Codigo_Item) AS Codigo_Item,
       c.Descripcion_Item,
       ISNULL(MAX(CASE WHEN h.Id_Correlativo_Lab = 1 THEN h.Valor_Lab END), '') AS LAB1,
       ISNULL(MAX(CASE WHEN h.Id_Correlativo_Lab = 2 THEN h.Valor_Lab END), '') AS LAB2,
       ISNULL(MAX(CASE WHEN h.Id_Correlativo_Lab = 3 THEN h.Valor_Lab END), '') AS LAB3,
       h.Fecha_Registro AS F_REGISTRO,
       h.Fecha_Modificacion AS F_MODIFICACION,            
       r.est_nombre AS ESTABLECIMIENTO,
       CONCAT(r.DESC_DIST,' | ', r.DESC_PROV) AS [DISTRITO | PROVINCIA],
       S.Descripcion_Sistema AS SISTEMA,
       LTRIM(RTRIM(
            COALESCE(re.Nombres_Registrador,'') + ' ' +
            COALESCE(re.Apellido_Paterno_Registrador,'') + ' ' +
            COALESCE(re.Apellido_Materno_Registrador,'')
       )) AS REGISTRADOR
FROM DBGERESA.dbo.HISMINSA h
INNER JOIN DBGERESA.dbo.MAESTRO_PACIENTE p
    ON p.Id_Paciente = h.Id_Paciente
INNER JOIN DBGERESA.dbo.RENIPRESS r
    ON r.COD_ESTAB = h.renipress
LEFT JOIN DBGERESA.dbo.MAESTRO_HIS_CIE_CPMS c
    ON c.Codigo_Item = h.Codigo_Item
LEFT JOIN DBGERESA.dbo.MAESTRO_HIS_SISTEMA S
    ON S.Id_Sistema = H.Id_AplicacionOrigen
LEFT JOIN DBGERESA.dbo.MAESTRO_REGISTRADOR re
    ON re.Id_Registrador = h.Id_Registrador
WHERE h.id_cita IN {resultado_str}
  AND h.Anio = 2025
  and h.Codigo_Item IN ('87621')
  and p.Genero='F'
GROUP BY
     h.id_cita,
    p.Numero_Documento ,
    h.Fecha_Atencion,
    h.Tipo_Diagnostico,
    h.Codigo_Item,
    c.Descripcion_Item,
    h.Fecha_Registro,
    h.Fecha_Modificacion,
    S.Descripcion_Sistema,
    r.est_nombre,
    r.DESC_DIST,
    r.DESC_PROV,
    re.Nombres_Registrador,
    re.Apellido_Paterno_Registrador,
    re.Apellido_Materno_Registrador
ORDER BY h.Fecha_Atencion DESC;"""
his_result=conn.ejecutar_sql(q,retornar_datos=True)
his_result=pd.DataFrame(his_result)
his_result['id_cita_r'] = pd.to_numeric(his_result["id_cita_r"], errors="coerce").astype("Int64")
his_result.head(5)

,id_cita_r,DNI_HIS,F_ATENCION,Codigo_Item,Descripcion_Item,LAB1,LAB2,LAB3,F_REGISTRO,F_MODIFICACION,ESTABLECIMIENTO,DISTRITO | PROVINCIA,SISTEMA,REGISTRADOR
0,1253344952,46083479,2025-07-13,D | 87621,TEST PVH (TOMA DE MUESTRA),N,,,2025-07-28 04:17:33,None,PAMPAPHALLA,SICUANI | CANCHIS,INFORMACION REMITIDA POR CANCER,PATRICIA JEANNET MENDOZA VELASQUEZ
1,1253345728,43749347,2025-07-11,D | 87621,TEST PVH (TOMA DE MUESTRA),N,,,2025-07-28 04:20:45,None,CCATCCA,CCATCA | QUISPICANCHI,INFORMACION REMITIDA POR CANCER,SILVIA VIRGINIA TAPIA FARFAN
2,1253345729,24716922,2025-07-11,D | 87621,TEST PVH (TOMA DE MUESTRA),N,,,2025-07-28 04:20:45,None,PAMPAPHALLA,SICUANI | CANCHIS,INFORMACION REMITIDA POR CANCER,PATRICIA JEANNET MENDOZA VELASQUEZ
3,1253345732,47483372,2025-07-11,D | 87621,TEST PVH (TOMA DE MUESTRA),N,,,2025-07-28 04:20:45,None,QUEHUAR,SICUANI | CANCHIS,INFORMACION REMITIDA POR CANCER,CONCEPCION CCARHUARUPAY TICA
4,1253345730,45402016,2025-07-11,D | 87621,TEST PVH (TOMA DE MUESTRA),N,,,2025-07-28 04:20:45,None,PAMPAPHALLA,SICUANI | CANCHIS,INFORMACION REMITIDA POR CANCER,SILVIA ROXANA ARAPA APAZA


In [ ]:

df_merge1 = vph.merge(
    his_toma,
    left_on="Hisminsa/tomamuestra/idCita",
    right_on="id_cita",
    how="inner"
)
df_final = df_merge1.merge(
    his_result,
    left_on="Hisminsa/resultado/idCita",
    right_on="id_cita_r",
    how="inner"
)

# Exportar a Excel
df_final.to_excel("Export/vph_odo_his.xlsx", index=False)
df_final.head(5)

,Establecimiento_Salud,Código_de_barras,Fecha_de_la_toma,Paciente,Resultado,Resultado/Fecha_de_Entrega,Hisminsa/tomamuestra/idCita,Hisminsa/resultado/idCita,Red_Salud,Microred,Status,DNI,id_cita,DNI_HIS_x,F_ATENCION_x,Codigo_Item_x,Descripcion_Item_x,LAB1_x,LAB2_x,LAB3_x,F_REGISTRO_x,F_MODIFICACION_x,ESTABLECIMIENTO_x,DISTRITO | PROVINCIA_x,SISTEMA_x,REGISTRADOR_x,id_cita_r,DNI_HIS_y,F_ATENCION_y,Codigo_Item_y,Descripcion_Item_y,LAB1_y,LAB2_y,LAB3_y,F_REGISTRO_y,F_MODIFICACION_y,ESTABLECIMIENTO_y,DISTRITO | PROVINCIA_y,SISTEMA_y,REGISTRADOR_y
0,C.S. ACCHA,CUS000023627,2025-05-17,CUTIPA HANCCO ANGELICA,NEGATIVO,2025-07-08,1216006514,1246315758,CUSCO SUR,PARURO,ER Hisminsa,44152466,1216006514,44152466,2025-05-12,D | 87621,TEST PVH (TOMA DE MUESTRA),1,,,2025-05-15 17:43:16,NaT,ACCHA,ACCHA | PARURO,INFORMACION REMITIDA POR CANCER,MARIA LISBETH QUISPE QUISPE,1246315758,44152466,2025-05-17,D | 87621,TEST PVH (TOMA DE MUESTRA),N,,,2025-07-12 12:05:28,None,ACCHA,ACCHA | PARURO,INFORMACION REMITIDA POR CANCER,MARIA LISBETH QUISPE QUISPE
1,C.S. ACCHA,CUS000023624,2025-05-15,DIAZ FERNANDEZ VIRGINIA,NEGATIVO,2025-06-30,1216006515,1243333096,CUSCO SUR,PARURO,ER Hisminsa,44152449,1216006515,44152449,2025-05-12,D | 87621,TEST PVH (TOMA DE MUESTRA),1,,,2025-05-15 17:43:16,NaT,ACCHA,ACCHA | PARURO,INFORMACION REMITIDA POR CANCER,MARIA LISBETH QUISPE QUISPE,1243333096,44152449,2025-05-15,D | 87621,TEST PVH (TOMA DE MUESTRA),N,,,2025-07-05 16:43:06,None,ACCHA,ACCHA | PARURO,INFORMACION REMITIDA POR CANCER,MARIA LISBETH QUISPE QUISPE
2,C.S. ACCHA,CUS000023623,2025-05-16,LOAIZA SENCIA VALENTINA,NEGATIVO,2025-06-30,1216006513,1243333097,CUSCO SUR,PARURO,ER Hisminsa,42455594,1216006513,42455594,2025-05-12,D | 87621,TEST PVH (TOMA DE MUESTRA),1,,,2025-05-15 17:43:16,NaT,ACCHA,ACCHA | PARURO,INFORMACION REMITIDA POR CANCER,MARIA LISBETH QUISPE QUISPE,1243333097,42455594,2025-05-16,D | 87621,TEST PVH (TOMA DE MUESTRA),N,,,2025-07-05 16:43:06,None,ACCHA,ACCHA | PARURO,INFORMACION REMITIDA POR CANCER,MARIA LISBETH QUISPE QUISPE
3,C.S. ACCHA,CUS000023621,2025-04-10,GARATE ORDOÑEZ LOURDES,NEGATIVO,2025-05-17,1202377168,1220243581,CUSCO SUR,PARURO,ER Hisminsa,48465973,1202377168,48465973,2025-04-10,D | 87621,TEST PVH (TOMA DE MUESTRA),1,,,2025-04-18 17:08:52,NaT,ACCHA,ACCHA | PARURO,INFORMACION REMITIDA POR CANCER,MARIA LISBETH QUISPE QUISPE,1220243581,48465973,2025-04-10,D | 87621,TEST PVH (TOMA DE MUESTRA),N,,,2025-05-25 00:53:03,None,ACCHA,ACCHA | PARURO,INFORMACION REMITIDA POR CANCER,MARIA LISBETH QUISPE QUISPE
4,C.S. ACCHA,CUS000023620,2025-04-09,HUAMAN LOPEZ YONY,NEGATIVO,2025-05-17,1202377169,1220243582,CUSCO SUR,PARURO,ER Hisminsa,47711273,1202377169,47711273,2025-04-09,D | 87621,TEST PVH (TOMA DE MUESTRA),1,,,2025-04-18 17:08:52,NaT,ACCHA,ACCHA | PARURO,INFORMACION REMITIDA POR CANCER,MARIA LISBETH QUISPE QUISPE,1220243582,47711273,2025-04-09,D | 87621,TEST PVH (TOMA DE MUESTRA),N,,,2025-05-25 00:53:03,None,ACCHA,ACCHA | PARURO,INFORMACION REMITIDA POR CANCER,MARIA LISBETH QUISPE QUISPE
